<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/refactoring_CF_appliances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas.io import gbq

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
#pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.simplefilter("ignore")

## Collaborative Filtering 추천 시스템

### 추천 모델 구현을 위한 데이터 전처리
- Big-query를 활용하여 로그 데이터와 item 데이터 import
- 한 제품을 6개 이상 구매한 데이터 제외
- 구매 정보가 없는 데이터(Null)를 0으로 대체

In [3]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'appliances%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 5
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=BZeP64HoR8Yc5XuKWr6kCgbNvdOYR7&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWhiLqSPuQNWTFlUfGwYEaENiCg4UAyyNcYTKq3Sw6tHed3y60xThhg


In [19]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'appliances%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [4]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
31208,533441221,2900569,1
41604,513819263,3701134,1
41605,562357140,3701162,1
41606,551858155,3701088,1
41607,538069266,3701323,1
...,...,...,...
133,512762567,2701286,5
132,562301215,2701706,5
131,515668794,3601537,5
141,565831130,2501453,5


In [5]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [6]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
384989212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429474454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433754231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440756116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450756413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566221288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566227497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566235555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_final_limit = df_fin_limit.fillna(0)

In [8]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [9]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481341715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
494312717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500246310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512363712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566094079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566122364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566144303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Collaborative Filtering 구현
- SVD 행렬 분해 방식으로 구현
    - 차원 수가 많은 데이터에 유리한 알고리즘
    - 희소 문제에 유리한 알고리즘

In [28]:
# Matrix 계산을 위해 pandas를 numpy로 변환
matrix_limit = df_filtered_limit.values

# 유저의 평균 구매횟수
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

# 각 유저의 구매횟수에서 평균 구매횟수를 뺌으로써 평균에서 벗어난 특이값을 남김
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [13]:
# 행렬 분해
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 1000)

In [14]:
# 0이 포함된 대칭행렬로 전환
sigma_limit = np.diag(sigma_limit)

In [29]:
# 분해한 행렬에서 특이값을 추출한 후 다시 하나로 복구함
dot_matrix_limit = np.dot(U_limit, sigma_limit)
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

# 앞서 뺐던 유저의 평균 구매 횟수를 다시 더함
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [18]:
# 유저별로 제품에 대한 구매 확률을 예측한 테이블

df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.000086,-0.000089,-0.000089,-0.000151,0.000011,-0.000089,-0.000268,-0.000087,-0.000165,-0.000185,...,0.000564,-0.000089,-0.000013,-0.000089,-0.000089,-0.000184,-0.000089,-0.000089,-0.000089,-0.000205
481341715,0.000045,-0.000030,-0.000030,-0.000025,-0.000012,-0.000030,0.000089,-0.000008,0.000079,-0.000038,...,-0.000220,-0.000030,-0.000002,-0.000030,-0.000030,-0.000071,-0.000030,-0.000030,-0.000030,-0.000050
494312717,0.000034,-0.000046,-0.000046,-0.000033,-0.000037,-0.000046,0.000465,-0.000205,0.000036,-0.000033,...,0.000183,-0.000046,0.000012,-0.000046,-0.000046,-0.000028,-0.000046,-0.000046,-0.000046,-0.000082
500246310,-0.000035,-0.000032,-0.000032,0.000026,-0.000101,-0.000032,0.000714,-0.000033,0.000063,-0.000026,...,0.000304,-0.000032,0.000011,-0.000032,-0.000032,-0.000019,-0.000032,-0.000032,-0.000032,0.000009
512363712,-0.000033,-0.000027,-0.000027,-0.000016,-0.000033,-0.000027,0.000474,0.000094,0.000055,0.000019,...,-0.000111,-0.000027,0.000004,-0.000027,-0.000027,0.000002,-0.000027,-0.000027,-0.000027,0.000022


In [20]:
# 유저 Id를 입력하면 추천 아이템이 출력되는 함수

def recommend_items(df_svd_preds, user_id, ori_items_df, ori_ratings_df, num_recommendations=5):
    
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 데이터 정렬 -> 구매 확률이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 item 데이터를 합친다. 
    user_history = user_data.merge(ori_items_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 유저가 이미 구매한 item을 제외하고 데이터 추출
    recommendations = ori_items_df[~ori_items_df['product_id'].isin(user_history['product_id'])]

    # 정렬된 추천 list와 item 정보를 merge하여 하나의 테이블로 만듦 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [21]:
already_rated, predictions = recommend_items(df_svd_preds_limit, 512363712, df_product, df_ratings_limit, 10)

In [22]:
predictions

,product_id,brand,category_code,Predictions
3052,10900233,polaris,appliances.kitchen.mixer,0.033835
2585,10900341,dauscher,appliances.kitchen.mixer,0.030975
479,35700021,apollo,appliances.kitchen.kettle,0.026707
218,3100618,lumme,appliances.kitchen.blender,0.024924
2091,3200540,redmond,appliances.kitchen.meat_grinder,0.022615
4273,6301922,elenberg,appliances.kitchen.kettle,0.021740
3909,11100266,unit,appliances.personal.scales,0.018473
2643,10900026,panasonic,appliances.kitchen.mixer,0.013993
353,29100011,gezatone,appliances.personal.massager,0.012742
3640,2900194,samsung,appliances.kitchen.microwave,0.011547


### 평가지표
- RMSE : 실제 데이터의 잠재 요인을 얼마나 잘 추출했는지 원본 데이터와의 오차로 비교함
- Base Line
    - 평균 구매 횟수로 채운 테이블과 원본 데이터와의 MSE를 산출하여 모든 사용자가 평균만큼은 구매할 것으로 예측하고 오차를 계산함
    - Score : 0.75
- Truncated SVD 
    - SVD로 예측된 테이블과 원본 데이터와의 MSE를 산출하여 추출된 잠재 요인만큼 상품을 구매할 것으로 예측하고 오차를 계산함
    - Score : 0.49


In [25]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [26]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.7565141480224759

In [27]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

0.49069946220778593